In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

%matplotlib inline

In [ ]:
# Set default colors
default_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                  '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                  '#bcbd22', '#17becf']

mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=default_colors)

# Default markers
mrk = ['o', '*', 'd', 's', '^', 'p', 'v']

## Load-displacement curves

In [ ]:
pd1 = pd.read_csv('../data/rad_buckling_experiments/BR17125839_Pd.csv', header=0)
pd2 = pd.read_csv('../data/rad_buckling_experiments/BR17282956_Pd.csv', header=0)
pd3 = pd.read_csv('../data/rad_buckling_experiments/BR17282957_Pd.csv', header=0)

In [ ]:
def trim_rad(pd, n_trim):
    'Trim and shift non-linear settling portion of load-displacement curve.'
    
    pd = pd.drop(index=range(n_trim))
    
    pf = np.polyfit(pd['EXT_RAD'][:40], pd['FORCE'][:40], 1)
    
    v_offset = -pf[1]/pf[0]
    
    pd['EXT_RAD'] = pd['EXT_RAD'] - v_offset
    
    return pd

# Trim non-linear settling portion at the beginning
pd1 = trim_rad(pd1, 5)
pd2 = trim_rad(pd2, 0)
pd3 = trim_rad(pd3, 5)

In [ ]:
kN = 0.00444822 # Conversion from lbs to kN

cm = sns.color_palette('Blues', 3)

with plt.style.context('seaborn-paper'):
    fig, ax = plt.subplots(ncols=2, figsize=(6.5, 3.25))

    ax[0].plot(pd1['EXT_RAD'], pd1['FORCE']*kN, '.:', color=cm[2])
    ax[0].plot(pd2['EXT_RAD'], pd2['FORCE']*kN, '.:', color=cm[1])
    ax[0].plot(pd3['EXT_RAD'], pd3['FORCE']*kN, 'C3.:')
    
    ax[0].set_ylim([0., 5.])
    ax[0].set_xlim([0., 12.])
    
    ax[0].set_xlabel('Radial displacement [mm]')
    ax[0].set_ylabel('Radial load [kN]')
    
    
    ax[1].plot(pd1['EXT_LAT'], pd1['EXT_RAD'], '.:', color=cm[2])
    ax[1].plot(pd2['EXT_LAT'], pd2['EXT_RAD'], '.:', color=cm[1])
    ax[1].plot(pd3['EXT_LAT'], pd3['EXT_RAD'], 'C3.:')
    
    ax[1].set_xlim([-5., 42.])
    ax[1].set_ylim([0., 12.])
    
    ax[1].set_xlabel('Lateral displacement [mm]')
    ax[1].set_ylabel('Radial displacement [mm]')
    
    plt.tight_layout()
    plt.savefig('../figs/buckling_ext_loads/_python_NU_experiments.pdf')

### Peak loads and strength difference

In [ ]:
print 'Wheel 1: {:.2f} kN ({:.1f} lbs)'.format(pd1['FORCE'].max()*kN, pd1['FORCE'].max())
print 'Wheel 2: {:.2f} kN ({:.1f} lbs)'.format(pd2['FORCE'].max()*kN, pd2['FORCE'].max())
print 'Wheel 3: {:.2f} kN ({:.1f} lbs)'.format(pd3['FORCE'].max()*kN, pd3['FORCE'].max())

print '\nStrength difference:'
print '{:.2f} kN ({:.1f} lbs)'.format(pd1['FORCE'].max()*kN - pd3['FORCE'].max()*kN,
                                    pd1['FORCE'].max() - pd3['FORCE'].max())

# Post-mortem: spoke tensions and deformation

In [ ]:
pm1 = pd.read_csv('../data/rad_buckling_experiments/BR17125839_post_mortem.csv', header=0)
pm2 = pd.read_csv('../data/rad_buckling_experiments/BR17282956_post_mortem.csv', header=0)
pm3 = pd.read_csv('../data/rad_buckling_experiments/BR17282957_post_mortem.csv', header=0)

In [ ]:
def c(x, i):
    'Shift elements of x so that the ith element is in the center'
    
    return np.roll(x, len(x)/2 - i)


def plot_wheel(pm, spk_load, ax1):
    Tdiff = c((pm['T_POST'] - pm['T_PRE']).tolist(), spk_load)
    lat_def = c((pm['LAT_DEF'] - pm['LAT_DEF'].mean()).tolist(), spk_load)
    
    ax1.bar(range(1, 37, 2), Tdiff[::2], color='C0', alpha=0.8)
    ax1.bar(range(2, 37, 2), Tdiff[1::2], color='C1', alpha=0.8)
    ax1.bar(18, Tdiff[17], color='C3')
    
    ax2 = ax1.twinx()
    ax2.plot(range(1, 37), -lat_def, color='#333333')
    
    
    ax1.set_xticks([])
    ax1.set_ylim([-1000, 1000])
    ax2.set_ylim([-10, 10])
    
    ax1.set_yticklabels([])
    ax2.set_yticklabels([])


with plt.style.context('seaborn-paper'):
    fig, ax = plt.subplots(ncols=3, figsize=(5.5, 2.25))
    
    plot_wheel(pm1, 6, ax[0])
    plot_wheel(pm2, 14, ax[1])
    plot_wheel(pm3, 14, ax[2])
    
    plt.tight_layout()
    plt.savefig('../figs/buckling_ext_loads/_python_NU_experiments_postmortem.pdf')